In [2]:
#Import Dependencies
import pandas as pd
import json

# Import the world countries internet data from the World Bank

In [4]:
# Import the countries internet data from world bank 
world_data = pd.read_csv('data/world_bank_internet_data.csv', skipfooter=446, engine='python')
# Replace all ".." with 0
world_data = world_data.replace("..",0)
#Rename the Country and Country Codes column to allow merging later on
world_data.rename(columns={'Country Name': 'Country', 'Country Code': 'Codes'}, inplace=True)
# Remove the brackets and the duplicated value in the Year columns 
world_data = world_data.rename(columns={col: col.split('[')[0] for col in world_data.columns})
# Remove space in Year columns
world_data.columns = world_data.columns.str.strip()
cols = [3,4,5,6,7,8,9,10,11,12]
world_data.drop(world_data.columns[cols],axis=1,inplace=True)


In [5]:
#Import the coordinate csv file, rename columns for merging with each other later on
coordinates = pd.read_csv('data/world_country.csv')
coordinates.rename(columns={'iso_con' : 'Codes', 'lat': 'Latitude', 'lon': 'Longitude'},inplace=True)
coordinates.drop(columns=['Unnamed: 0', 'country'],inplace=True)


In [6]:
# Merge the coordinates with tHE world_data DataFrame
final_world = pd.merge(world_data, coordinates, on='Codes', how='left')
# Find out if there are any NaN values for Latitude and Longitude and then drop them 
final_world[final_world.isnull().any(axis=1)] 
final_world.dropna(subset=['Latitude','Longitude'],inplace=True)


In [7]:
# Turn the Year columns into Rows
data = pd.melt(final_world, id_vars=['Country', 'Codes','Series Name', 'Longitude', 'Latitude'], var_name='Year', value_name='Values')
# Put all the values in the Values column to numeric
data['Values'] = pd.to_numeric(data['Values'])


In [ ]:
# Get the unique Series Name column values
print(data['Series Name'].unique())

In [8]:
# Creating dataframes from each unique value in Series Name column
world1=data[data['Series Name']=='Individuals using the Internet (% of population)'].copy()
world2=data[data['Series Name']=='Secure Internet servers'].copy()
world3=data[data['Series Name']=='Secure Internet servers (per 1 million people)'].copy()
world4=data[data['Series Name']=='Mobile cellular subscriptions'].copy()
world5=data[data['Series Name']=='Fixed broadband subscriptions'].copy()
world6=data[data['Series Name']=='Fixed broadband subscriptions (per 100 people)'].copy()
world7=data[data['Series Name']=='Fixed telephone subscriptions'].copy()
world8=data[data['Series Name']=='Fixed telephone subscriptions (per 100 people)'].copy()

# Dropping Series Name column and renaming values column to the 8 unique values we extracted from Series Name column
world1.drop(columns=['Series Name'],inplace=True)
world1.rename(columns={'Values': 'Individuals using the Internet (% of population)'}, inplace=True)
world2.drop(columns=['Series Name'],inplace=True)
world2.rename(columns={'Values': 'Secure Internet servers'}, inplace=True)
world3.drop(columns=['Series Name'],inplace=True)
world3.rename(columns={'Values': 'Secure Internet servers (per 1 million people)'}, inplace=True)
world4.drop(columns=['Series Name'],inplace=True)
world4.rename(columns={'Values': 'Mobile cellular subscriptions'}, inplace=True)
world5.drop(columns=['Series Name'],inplace=True)
world5.rename(columns={'Values': 'Fixed broadband subscriptions'}, inplace=True)
world6.drop(columns=['Series Name'],inplace=True)
world6.rename(columns={'Values': 'Fixed broadband subscriptions (per 100 people)'}, inplace=True)
world7.drop(columns=['Series Name'],inplace=True)
world7.rename(columns={'Values': 'Fixed telephone subscriptions'}, inplace=True)
world8.drop(columns=['Series Name'],inplace=True)
world8.rename(columns={'Values': 'Fixed telephone subscriptions (per 100 people)'}, inplace=True)

#Resetting Index
world1.reset_index(drop=True, inplace=True)
world2.reset_index(drop=True, inplace=True)
world3.reset_index(drop=True, inplace=True)
world4.reset_index(drop=True, inplace=True)
world5.reset_index(drop=True, inplace=True)
world6.reset_index(drop=True, inplace=True)
world7.reset_index(drop=True, inplace=True)
world8.reset_index(drop=True, inplace=True)

# Merge the 8 Dataframes into one
world = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(world1,world2), world3), world4), world5),world6),world7),world8)

# Import the region part of the World Bank Data csv file

In [ ]:
# #Import the regions data from csv file
# world_regions = pd.read_csv('data/world_bank_internet_data.csv', skiprows=range(1,1954), skipfooter=5, engine='python')
# # Replace all ".." with 0
# world_regions = world_regions.replace("..",0)
# # #Rename the Country and Country Codes columns
# world_regions.rename(columns={"Country Name": "Region"}, inplace=True)
# # # Remove the brackets and the duplicated value in the Year columns 
# world_regions = world_regions.rename(columns={col: col.split('[')[0] for col in world_regions.columns})
# world_regions.drop(columns=['Series Code', 'Country Code'], axis=1, inplace=True)
# cols = [2,3,4,5,6,7,8,9,10]
# world_regions.drop(world_regions.columns[cols],axis=1,inplace=True)
# world_regions

In [ ]:
# print(world_regions['Region'].unique())

In [ ]:
#   data_path = os.path.join(os.path.dirname(__file__), '..', 'data', "world_bank_internet_data.csv")
#     world_regions = pd.read_csv(data_path, skiprows=range(1,1954), skipfooter=5, engine="python")
#     # Replace all ".." with 0
#     world_regions = world_regions.replace("..",0)
#     # #Rename the Country and Country Codes columns
#     world_regions.rename(columns={"Country Name": "Country", "Country Code": "Codes"}, inplace=True)
#     # # Remove the brackets and the duplicated value in the Year columns 
#     world_regions = world_regions.rename(columns={col: col.split('[')[0] for col in world_regions.columns})
#     world_regions.drop(columns=['Series Code', 'Codes'], axis=1, inplace=True)
#     world_regions.drop(world_regions.columns[cols],axis=1,inplace=True)

# Import ITU Regional, Age and Gender Internet Usage Data

### Regions

In [9]:
# Reading the ITU excel file region sheet to clean and turn into json object
temp_df = pd.read_excel('data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', header=None, sheet_name = "By BDT region")

# Getting the first table
temp_df.iloc[3:10,0:18]
# Getting the first column names
table_name = temp_df.iloc[2,0]

#dictionary of tables with key as table name and value as dataframe
table_list = {}
# year list
year_list = list(range(2005,2022))
# range start at 4(3), all the way to row 92 with a step of 9
for i in range(4,92,9):
    #table name, 2 rows from where the table start at 0 (1st) column
    table_name = temp_df.iloc[i-2,0]
    # selecting range of rows and range of columns to extract a table into a dataframe 
    region_df = temp_df.iloc[i:i+6,0:18]
    region_df.reset_index(drop=True, inplace=True)
    # Using year_list and table_name as column names
    column_names= [table_name] + year_list
    # Inserting column names
    region_df.columns = column_names
    table_list[table_name]= region_df

In [13]:
region_df

,Individuals using the Internet,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Africa,14.5,19.1,23.8,28.5,34.4,49.8,64.2,80.7,100.9,124.1,153.2,176.3,209.8,246.2,278.6,315.4,360.1
1,Americas,315,344.6,383.6,401.5,424.0,451.9,478.8,521.3,535.5,561.7,607.9,668.7,711.1,740.7,770.4,802.2,834.7
2,Arab States,27.3,34.0,48.2,56.8,67.5,79.8,91.1,104.3,117.4,132.9,146.8,172.2,199.4,214.2,239.7,270.9,291.1
3,Asia-Pacific,361,413.6,524.4,635.6,748.2,894.4,1008.6,1103.7,1203.9,1302.1,1404.4,1533.9,1635.2,1804.7,2076.7,2418.4,2613.3
4,CIS,24.7,31.8,42.6,55.0,74.0,81.1,96.0,122.0,133.2,142.3,146.6,155.3,162.3,175.0,185.6,193.3,200.7
5,Europe,279.6,302.1,343.0,372.4,395.9,421.2,433.1,451.5,467.4,483.6,492.0,507.8,523.1,546.3,565.2,582.5,598.9


### Gender

In [10]:
# Reading the ITU excel file sex and age sheet to clean and turn into gender and age dataframes for data cleaning
gender_df = pd.read_excel('data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', header=[3,4], sheet_name = "Internet use by sex & age")

In [ ]:
# Get the gender tables
gender_df=gender_df.iloc[:13,0:12]
#Drop unnecessary columns
cols = [4,8]
gender_df.drop(gender_df.columns[cols],axis=1,inplace=True)
# Drop unnecessary rows
gender_df = gender_df.drop(6)
gender_df = gender_df.drop(7)

### Age

In [12]:
age_df = pd.read_excel('data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', skiprows=range(0,19),skipfooter=10 , sheet_name = "Internet use by sex & age")


In [ ]:
# Get the age tables from the gender_age excel file
age_df=age_df.iloc[18:32,0:7]
# cols = [4,5]
# age_df.drop(age_df.columns[cols],axis=1,inplace=True)
# age_df = age_df.drop(24)
# age_df = age_df.drop(25)
age_df